# Projeto SiDi - Resolução de Entidade

## Scraper

In [156]:
import requests
import pandas as pd
import re
import time

from bs4 import BeautifulSoup

In [153]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            #'Accept-Language': 'en-US, en;q=0.5',
            "Accept-Encoding":"gzip, deflate",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "DNT":"1","Connection":"close",
            "Upgrade-Insecure-Requests":"1"})

COOKIES = {'i18n-prefs': "BRL",
           'session-id': "147-4481349-3776115",
           'session-id-time': "2082758401l",
           'session-token': "Z6ZJVobSCATkyNn1g8kPOtbqWGyda/cvfli854KW3r8ywSWxPTbbMDNsGEUQsqEO3WZtuwGPS9A10PxeR9gVF0rFIGj2f0dnoJ7flEKVDce9nJcIbldcT2P2BZlkPBIKr+YEP2ErDo8NSmXjvOIrT2PeUH5op46fQR9EGQsrBrM38pnVw74esum4cEdTpTNgPjBQ4Uw939qsJJjs8V+mAw==",
           'ubid-acbbr': "133-1643912-7168465",
           'csm-hit': "tb:QTSKC9XAV6CW3SJ7NA4T+s-QTSKC9XAV6CW3SJ7NA4T|1651786046763&t:1651786046763&adb:adblk_yes"
           }

def getdata(url):
    r = requests.get(url, headers = HEADERS, cookies = COOKIES)
    return r.text

def html_code(url):
  
    # pass the url
    # into getdata function
    htmldata = getdata(url)
    soup = BeautifulSoup(htmldata, 'html.parser')
  
    # display html code
    return (soup)

In [3]:
def cus_data(soup):
    # find the Html tag
    # with find()
    # and convert into string

    cus_list= []

    for item in soup.find_all("div", class_="a-section celwidget"):
        review = item.find("span", class_="a-size-base review-text review-text-content").get_text()
        date = item.find("span", class_="a-size-base a-color-secondary review-date").get_text()
        stars = item.find("span", class_="a-icon-alt").get_text()
        
        cus_list.append((review, stars, date))

    return cus_list

In [31]:
'''links = []
for paginacao in range(1, 173):
    
    url = f"https://www.amazon.com.br/s?i=electronics&rh=n%3A16243890011&s=price-desc-rank&fs=true&page={paginacao}&qid=1651783337&ref=sr_pg_{paginacao}"
    soup = html_code(url)

    lista_divs = soup.find_all("div", {"class": "a-section a-spacing-none a-spacing-top-small s-title-instructions-style"})

    for div in lista_divs:
        for link in div.findAll('a', attrs={'href': re.compile("^/")}):
            links.append(f"https://www.amazon.com.br{link.get('href')}")
            
links = list(set(links))
'''

In [33]:
#df = pd.DataFrame(links, columns = ['links'])
#df.to_csv('Dados/amazon_links_.csv', index = False)

In [209]:
df_links = pd.read_csv('Dados/amazon_links.csv')

In [210]:
lista_produtos = []
for i, url in enumerate(df_links['links']):
    
    soup = html_code(url)
    
    titulo = soup.find("span", {"class": "a-size-large product-title-word-break"}).text.strip()
    tabela = soup.find("table", {"class": "a-keyvalue prodDetTable"})
    ean = re.findall(r'EAN <\/th> <td class="a-size-base prodDetAttrValue">\n                \u200e([0-9]+)|$', str(tabela))[0]
    lista_produtos.append((url, titulo, ean))
    
    if i >= 5:
        break
    
    time.sleep(10)

In [212]:
df_produtos = pd.DataFrame(lista_produtos, columns = ['url', 'titulo', 'ean'])

In [213]:
df_produtos['titulo'] = df_produtos['titulo'].astype('string')
df_produtos['ean'] = df_produtos['ean'].astype('string')

In [215]:
df_produtos.to_csv("Dados/produtos_amazon.csv", index = False)